In [1]:
# GPU BEGONE!
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu'
os.environ['PATH'] = '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ

environ{'USER': 'fandav',
        'JUPYTERHUB_HOST': '',
        'LD_LIBRARY_PATH': '/usr/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu',
        'JUPYTERHUB_USER': 'fandav',
        'SHLVL': '0',
        'HOME': '/home/fandav',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/fandav/oauth_callback',
        'SLURM': '1',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-fandav',
        'PATH': '/home/fandav/.local/bin:/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/opt/anaconda3/envs/jupyter/bin:/opt/anaconda3/bin/:/snap/bin',
        'JUPYTERHUB_ACTIVITY_URL': 'http://127.0.0.1:8081/hub/api/users/fandav/activity',
        'LANG': 'en_US.UTF-8',
        'SHELL': '/bin/zsh',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/fandav/',
        'PWD': '/home/fandav',
        'JUPYTERHUB_API_TOKEN': 'fce9f2baebfc44f795960c829286241b',
        'JUPYTERHUB_BASE_URL': '/',
        'XDG_DATA_DIRS': '/usr/local/share:/usr/share:/v

In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
import pandas as pd

# To find out which devices your operations and tensors are assigned to
#tf.debugging.set_log_device_placement(True)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

# Obtain Embeddings

In [3]:
# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased');

# model
model = TFBertModel.from_pretrained('bert-base-uncased');
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


# Import Data

In [4]:
dataset = pd.read_csv('final_dataset.csv', index_col=0)
#display(dataset.loc['Content'].values)

In [5]:
# grab content
dataset_content = dataset.loc['Content'].fillna('').values
#print(dataset_content)

# process content through BERT
encoded_strings = [tokenizer.encode(content, max_length=512, pad_to_max_length=True) for content in dataset_content]
encoded_vectors = tf.constant(encoded_strings)
outputs = model(encoded_vectors)

# obtain per line embeddings from all the artibles from BERT
last_hidden_states = outputs[0]

# obtain article embeddings from BERT
article_embeddings = outputs[1]

print(np.shape(outputs))
print(np.shape(outputs[0]))
print(np.shape(outputs[1]))

(2, 50)
(50, 512, 768)
(50, 768)


In [6]:
_, num_articles = np.shape(outputs)
article_names = [f'Article {idx}' for idx in range(num_articles)]

# vector embeddings of each article
vec_embeddings = article_embeddings.numpy()

In [20]:
# final classifier labels
credibility_label = np.array(list(map(int, dataset.loc['Credibility Score'].values)))

# run NN with last layers changed to fit labels for each indicator
indicators = [
              'Originality Score',
              'Fact-Checked',
              'Attribution Rank',
              'Source 1 Category',
              'Source 2 Category',
              'Source 3 Category',
              'Article has Spammy/Clickbaity Ads',
              'Article has Aggressive Ads/Calls'
             ]

indicator_data = []
for truth_label in indicators:
    label_values = np.array(np.array(list(map(int, dataset.loc[truth_label].values))))
    indicator_data.append([label_values, len(label_values), set(label_values), len(set(label_values))])

In [21]:
indicators_df = pd.DataFrame(indicator_data,
                             columns=['truth label values', 'number of truth label values', 'unique truth labels', 'size of unique truth labels'],
                             index=indicators
                            )
display(indicators_df)

,truth label values,number of truth label values,unique truth labels,size of unique truth labels
Originality Score,"[0, 2, 0, 1, 3, 1, 0, 2, 0, 2, 2, 3, 0, 1, 2, ...",50,"{0, 1, 2, 3}",4
Fact-Checked,"[1, 2, 4, 1, 2, 1, 1, 2, 3, 1, 2, 1, 1, 1, 3, ...",50,"{0, 1, 2, 3, 4}",5
Attribution Rank,"[1, 2, 0, 1, 3, 1, 0, 2, 0, 2, 2, 3, 0, 4, 3, ...",50,"{0, 1, 2, 3, 4}",5
Source 1 Category,"[0, 2, 2, 0, 1, 3, 2, 2, 2, 0, 2, 2, 0, 3, 2, ...",50,"{0, 1, 2, 3}",4
Source 2 Category,"[0, 1, 0, 0, 1, 3, 3, 2, 3, 0, 1, 2, 0, 3, 0, ...",50,"{0, 1, 2, 3}",4
Source 3 Category,"[0, 1, 1, 0, 1, 3, 1, 2, 3, 0, 2, 2, 0, 3, 0, ...",50,"{0, 1, 2, 3}",4
Article has Spammy/Clickbaity Ads,"[2, 3, 1, 3, 2, 4, 4, 3, 4, 4, 2, 1, 2, 4, 3, ...",50,"{0, 1, 2, 3, 4, 5}",6
Article has Aggressive Ads/Calls,"[3, 3, 2, 4, 2, 3, 4, 4, 3, 4, 2, 1, 2, 4, 2, ...",50,"{0, 1, 2, 3, 4, 5}",6


In [8]:
display(dataset)

,Article 43,Article 13,Article 14,Article 30,Article 23,Article 42,Article 49,Article 36,Article 45,Article 7,...,Article 44,Article 46,Article 35,Article 38,Article 37,Article 28,Article 29,Article 39,Article 41,Article 12
Title,Which Banana Would You Eat? Your Answer May Ha...,NOW IT’S OFFICIAL: FDA Announced That Vaccines...,CONFIRMED: E-CIGARETTES CAUSE A HORRIBLE INCUR...,WITH ONLY 2 CUPS A DAY FOR 1 WEEK YOUR STOMACH...,"Lead Developer of HPV Vaccines Comes Clean, Wa...",DELINGPOLE: Global Warming Study Cancelled Bec...,Diet drinks TRIPLE your risk of stroke and dem...,NaN,NaN,Nations Press: PLS SHARE: DO NOT EAT THIS FISH...,...,NaN,Opinion | How the Anti-Vaxxers Are Winning,Arctic’s Winter Sea Ice Drops to Its Lowest Re...,The Best Exercise for Aging Muscles,An Iceberg the Size of Delaware Just Broke Awa...,Putting Kids To Bed Early Improves Mom's Healt...,If Everyone Ate Beans Instead of Beef,Arctic stronghold of world’s seeds flooded aft...,"The oldest child is actually the smartest, stu...",Coconut oil isn't healthy. It's never been hea...
Content,"When we eat bananas, we do so because we are h...",The FDA has published conclusive proof on thei...,CONFIRMED: E-CIGARETTES CAUSE A HORRIBLE INCUR...,The desire of lots of people is to have a flat...,"Lead Developer of HPV Vaccines Comes Clean, Wa...",A global warming research study in Canada has ...,"The Boston University study of almost 4,400 ad...",NaN,NaN,NaN,...,NaN,A major measles outbreak in America is only a ...,Much of the ice also appears to be thinner tha...,Certain kinds of exercise may mitigate the eff...,A crack more than 120 miles long had developed...,A new study confirms why all those bedtime bat...,"With one dietary change, the U.S. could hypoth...",No seeds were lost but the ability of the rock...,"Birth order doesn't affect personality, but it...","“We advise against the use of coconut oil,"" th..."
URL,http://goodfullness.net/which-banana-would-you...,http://inshapetoday.com/now-official-fda-annou...,http://nowcheckthis.com/2017/03/23/confirmed-e...,http://publichealthabc.com/2-cups-day-1-week-s...,http://www.alternativenewsnetwork.net/lead-dev...,http://www.breitbart.com/big-government/2017/0...,http://www.dailymail.co.uk/~/article-4429790/i...,http://www.iflscience.com/environment/heavy-mo...,http://www.iflscience.com/health-and-medicine/...,http://www.nationspressph.com/2017/02/pls-shar...,...,https://www.ntd.tv/inspiring/parenting/sam-ber...,https://www.nytimes.com/2017/02/08/opinion/how...,https://www.nytimes.com/2017/03/22/climate/arc...,https://www.nytimes.com/2017/03/23/well/move/t...,https://www.nytimes.com/interactive/2017/06/09...,https://www.simplemost.com/new-study-says-putt...,https://www.theatlantic.com/health/archive/201...,https://www.theguardian.com/environment/2017/m...,https://www.today.com/health/birth-order-first...,https://www.usatoday.com/story/news/nation-now...
Credibility Score,1,0,1,0,0,1,1,0,1,0,...,2,4,4,3,4,2,4,4,3,3
Originality Score,0,2,0,1,3,1,0,2,0,2,...,2,0,0,0,0,0,0,0,0,0
Attribution Rank,1,2,0,1,3,1,0,2,0,2,...,3,0,0,0,0,0,0,0,0,0
Source 1 Category,0,2,2,0,1,3,2,2,2,0,...,3,2,2,2,3,2,3,3,2,2
Source 2 Category,0,1,0,0,1,3,3,2,3,0,...,3,3,2,1,2,2,2,2,2,2
Source 3 Category,0,1,1,0,1,3,1,2,3,0,...,3,1,0,0,1,2,2,2,1,3
Article has Spammy/Clickbaity Ads,2,3,1,3,2,4,4,3,4,4,...,3,1,0,0,0,4,1,1,3,2


# Define Neural Network Model

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalMaxPooling1D, Conv1D, MaxPooling1D, InputLayer, Input, Softmax

In [37]:
def model_def(last_layer_size):
    model = Sequential([
        Input(shape=(768, 1), name='input'),
        
        Conv1D(400, 3, padding='valid', activation='relu', strides=3, input_shape=(768,)),
        Dense(200, activation='relu', name='dense_0'),
        Dropout(0.2),
        
        #Conv1D(200, 3, padding='valid', activation='relu', strides=3),
        #Dense(150, activation='relu', name='dense_1'),
        #Dropout(0.2),
        
        Conv1D(100, 3, padding='valid', activation='relu', strides=5),
        GlobalMaxPooling1D(),
        
        Dense(64, activation='relu', name='dense_2'),
        
        #Dense(len(set(truth_labels)), name='dense_3', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dense(last_layer_size, name='dense_3', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Softmax()
    ]);
    #model.summary()

    model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
                  # Loss function to minimize
                  #loss=tf.keras.losses.SquaredHinge(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  # List of metrics to monitor
                  metrics=['accuracy']);
    return model

In [38]:
def train_model(model, x, y, epochs=300, batch_size=10, verbose=1):
    model.fit(x, y, batch_size=batch_size, epochs=epochs, verbose=verbose)
    return model

def test_model(model, x, y, verbose=1):
    test_loss, test_acc = model.evaluate(x,  y, verbose=verbose)
    return test_loss, test_acc

def predict(model, x):
    # predict a few samples
    output = model.predict(x)
    print(f'Output Vector: \n{output}')
    print(f'Output Vector Rounded: \n{np.round(output)}')
    print('Output Label: {}'.format([np.where(r==1)[0][0] for r in np.round(output)]))
    return output

In [53]:
class EndingLossAccCallback(tf.keras.callbacks.Callback):

    def on_train_batch_end(self, batch, logs=None):
        print('Training: batch {}, Loss: {:.2f}, Accuracy: {:.2f}'.format(batch,  logs['loss'], logs['accuracy']))

    def on_test_batch_end(self, batch, logs=None):
        print('Testing: batch {}, Loss: {:.2f}, Accuracy: {:.2f}'.format(batch,  logs['loss'], logs['accuracy']))

In [57]:
# sample test run
n_epochs = 100

    
train_index = np.arange(1, 50)
test_index = np.arange(1)

# input embeddings are the same for each indicator
X_train = vec_embeddings[train_index]
X_test = vec_embeddings[test_index]

# expand input to fit input layer
X_train = np.expand_dims(X_train, 2)
X_test = np.expand_dims(X_test, 2)
print(np.shape(X_train))

# there's probably a better way to do this -_-
indicator_models = {}
indicator_Y_train = {}
indicator_Y_test = {}

# generate a neural network model for each indicator
for indicator in indicators:
    output_layer_length = indicators_df.loc[indicator]['size of unique truth labels']
    truth_labels = indicators_df.loc[indicator]['truth label values']
    
    # output labels are based on indicator
    Y_train = truth_labels[train_index]
    Y_test = truth_labels[test_index]

    # define model for indicator
    model = model_def(output_layer_length)
    model.summary()
    
    # store handle to model and data as dict
    indicator_models[indicator] = model
    indicator_Y_train[indicator] = Y_train
    indicator_Y_test[indicator] = Y_test

# TRAIN ALLL THE MODELS!!!
for indicator in indicators:
    model = indicator_models[indicator]
    Y_train = indicator_Y_train[indicator]
    history = model.fit(X_train, Y_train,
                        batch_size=10,
                        epochs=n_epochs, verbose=0,
                        callbacks=[EndingLossAccCallback()])

(49, 768, 1)
Model: "sequential_96"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_192 (Conv1D)          (None, 256, 400)          1600      
_________________________________________________________________
dense_0 (Dense)              (None, 256, 200)          80200     
_________________________________________________________________
dropout_133 (Dropout)        (None, 256, 200)          0         
_________________________________________________________________
conv1d_193 (Conv1D)          (None, 51, 100)           60100     
_________________________________________________________________
global_max_pooling1d_96 (Glo (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_3 (Dense)              (None, 4)  

KeyboardInterrupt: 

In [12]:
# apply testing set
test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=1)
print(test_acc)

# predict a few samples
output = model.predict(X_train[0:20])
print(f'Output Vector: \n{output}')
print(f'Output Vector Rounded: \n{np.round(output)}')
print('Output Label: {}'.format([np.where(r==1)[0][0] for r in np.round(output)]))
print('Output Label: {}'.format(Y_train[0:20]))

1/1 [==============================] - 0s 809us/step - loss: 0.7453 - accuracy: 1.0000
1.0
Output Vector: 
[[9.99995112e-01 4.89134254e-06 9.17543493e-13 1.84507565e-09]
 [9.99999166e-01 7.81061885e-07 1.11542817e-12 1.37697276e-09]
 [9.99995351e-01 4.60555702e-06 8.41760635e-13 1.66954417e-09]
 [9.99998927e-01 1.07666517e-06 4.31751206e-12 3.07028070e-09]
 [9.99999762e-01 2.72568542e-07 5.14497130e-14 2.18451410e-10]
 [9.99999642e-01 2.99846789e-07 7.95574815e-14 2.90895280e-10]
 [9.25966788e-08 7.67067715e-04 9.99228358e-01 4.50005518e-06]
 [9.25966788e-08 7.67067715e-04 9.99228358e-01 4.50005518e-06]
 [9.25966788e-08 7.67067715e-04 9.99228358e-01 4.50005518e-06]
 [9.99995828e-01 4.15138993e-06 3.38267652e-12 3.74361608e-09]
 [9.99997616e-01 2.44372018e-06 1.15519280e-12 1.82320536e-09]
 [9.99999762e-01 2.10100822e-07 3.83975925e-14 1.63742103e-10]
 [9.99999523e-01 4.96138227e-07 1.98381730e-13 5.26731159e-10]
 [9.99999166e-01 7.90294393e-07 1.74759255e-13 4.97712538e-10]
 [9.9999988

In [13]:
from sklearn.model_selection import KFold

n_folds = 50
n_epochs = 1500
test_accuracy = []

kf = KFold(n_splits=n_folds)
kf.get_n_splits(vec_embeddings)
for k_index, (train_index, test_index) in enumerate(kf.split(vec_embeddings)):
    X_train = vec_embeddings[train_index]
    Y_train = truth_labels[train_index]
    
    X_test = vec_embeddings[test_index]
    Y_test = truth_labels[test_index]
    
    # expand input
    X_train = np.expand_dims(X_train, 2)
    X_test = np.expand_dims(X_test, 2)

    # reset model
    model = model_def()
    
    # apply training set
    history = model.fit(X_train, Y_train,
                        batch_size=10,
                        epochs=n_epochs, show=n_epochs%80==0, verbose=0);
                        # We pass some validation for
                        # monitoring validation loss and metrics
                        # at the end of each epoch
                        #validation_data=(x_val, y_val))
    
    # apply testing set
    test_loss, test_acc = model.evaluate(X_test,  Y_test, verbose=1)
    
    # record accuracy on testing
    test_accuracy.append(test_acc)
    
    print(f'Running Fold Number {k_index}')
    print(f'Size of Training Set: {len(train_index)}')
    print(f'Size of Testing Set: {len(test_index)}: {test_index}')
    print(f'Test accuracy: {test_acc}')
    print('------------------------')
    #model.predict(X_test[0:1])

TypeError: model_def() missing 1 required positional argument: 'last_layer_size'

In [ ]:
average_accuracy = np.mean(test_accuracy)

print(f'Average accuracy from a {n_folds}-Fold test: {average_accuracy}')